# FILT : Travail préparatoire du TP  
--- 

Le [TD FILT prepa_bilineaire](./FILT_prepa_bilineaire.ipynb) présente l'approximation bilinéaire discrète d'un dérivateur continu $p$

Ce travail préparatoire consiste à : 
   - passer d'une F.T. $G(p)$ continue à une F.T. discrète $G_d(z)$ fréquentiellement proche
   - savoir vérifier ses calculs par homogénéité et estimation géométrique de l'asymptotique
   - passer de la fonction de transfert en Z à une récurrence
   - savoir vérifier cette récurrence par homogénéîté et par son gain statique.

  

## 1 - Retrouver la formule et la méthode
---

On peut synthétiser un filtre discret en partant d'un filtre synthétisé en continu.  

Dans ce TP on propose de synthétiser un filtre du second ordre (le premier ordre est vu dans le [TD SYS, exo AR](../td/SYS_sujet.ipynb)) en partant de son expression continue :

$G(p) = K.\frac{\omega_n^2}{p^2+ 2\zeta\omega_n\,p+{\omega_n}^2}$

On a vu dans [TD FILT prepa_bilineaire](./FILT_prepa_bilineaire.ipynb) que la dérivée "Trapezoidale" de fonction de transfert $G_t(z)$ s'approche efficacement de la dérivée continue $p$ pour des fréquences suffisamment basses.

Il suffit alors de :
 1) prendre une fonction de transfert $G(p)$ dont le fréquentiel $G(\omega)=G(p\equiv j\omega)$ satisfait les fonctions du filtre
 2) remplacer $p$ par $G_t(z)$ dans $G(p)$ pour obtenir une fonction de transfert discrète $G_d(z) = G(p=G_t(z))$
 3) nous avons vu que la dérivée trapezoidale est appréciée, car son fréquentiel $G_t(\omega)=G_t(z\equiv e^{i2\pi\tilde{f}})\approx j\,\omega \equiv p$ pour de basses fréquences 
 4) ainsi $G_d(\omega) = G\left(p= G_t(z\equiv e^{i\,T_e\omega})\right)\approx G(\omega)=G(p\equiv j\omega)$ pour des fréquences suffisamment basses
 5) On passe classiquement de $G_d(z)$ à une récurrence $y[k] = y[k-\ldots + x\ldots$ permettant d'implanter le filtre dans un calculateur temps réel (du type STM32 ou PC, etc.)
 
> En se rappelant que 
>   - la dérivée est une pente : différence entre l'entrée $x$ et celle d'avant (incrément d'ordonnées) divisé par l'intervale des abscisses (le temps)
>   - la dérivée bilinéaire fait en sorte que la moyenne des dérivées $y$ aux instants $k-1$ et $k$ vaut la pente entre $k-1$ et $k$
>   - cette moyenne des sorties fait que le système discret est ARMA et pas juste MA
>   - cette dérivée est causale  
> **Identifiez quelle(s) formulation(s) sont bonnes** :  
> $G_t(z) =$ $\frac{z-1}{T_e}$ et/ou $\frac{1- z^{-1}}{T_e}$ et/ou $\frac{T_e}{z-1}$ et/ou $\frac{T_e}{1- z^{-1}}$ ?  
> $G_t(z) =$ $\frac{2}{T_e}\frac{z-1}{z+1}$ et/ou $\frac{2}{T_e}\frac{1+z}{1-z}$ et/ou $\frac{2}{T_e}\frac{z^{-1}-1}{z^{-1}+1}$ et/ou $\frac{2}{T_e}\frac{1-z^{-1}}{1+z^{-1}}$ ?  
> $G_t(z) =$ $\frac{T_e}{2}\frac{z-1}{z+1}$ et/ou $\frac{T_e}{2}\frac{1+z}{1-z}$ ?  
> $G_t(z) = e^{-Te.p}$ ?

## 2- Fonction de transfert continue
---

On veut obtenir un filtre discret proche du filtre continu du second ordre :

$G(p) = \frac{K}{p^2+ 2\zeta\omega_n\,p+{\omega_n}^2}$

On suppose que les signaux sont en Volts et le temps en secondes. 

 1) Donnez les dimensions de $G(p)$, $X(p)$, $Y(p)$ des paramètres $K$, $\omega_n$ et $\zeta$ et de $p$ et  
    **vérifiez l'homogénéité** de l'expression (**à corriger si problème !**).
 2) Calculez l'expression des pôles de $G$ et représentez-les dans le plan complexe.   
    Passez par la forme canonique $(p-\alpha)^2 + \omega_p^2 = 0$ plutôt que d'appliquer religieusement le Delta bachôté !  
    On en déduit les deux pôles $p_1, p_2 = \alpha \pm i \omega_p$ (lorsque $\omega_p^2$ est positif)  
    Vous devez retrouver les résultats de [cette figure](https://i.stack.imgur.com/J0Ste.png)
 4) Tracez le lieu de l'équivalent harmonique de $p$ sur ce plan complexe (le fameux $p\equiv j\omega$).  
 5) Ajoutez le tracé des vecteurs ${p-p_1}$ et ${p-p_2}$ dans ce plan complexe (comme en TD pour $z$). 
 3) Où sont les deux pôles $p_1$ et $p_2$ lorsque $\zeta=0$ ? 
 6) Dans ce cas, pour quelles fréquences ces vecteurs sont-ils de longueur nulle ? Quelle est alors le gain de la fonction de transfert ?  
    Cela fait résonner et raisonner !


## 3 - Fonction de transfert discrète
---
D'abord aprenons à vérifier des calculs, puis faisons le calcul en vérifiant à chaque ligne que l'on ne vient pas d'introduire une erreur.

### Méthodes de vérifications

Apprenez d'abord à vérifier vos calculs avec l'homogénéité :
> Donnez la dimension de $z$ et de $G_d(z)=\frac{Y(z)}{X(z)}$ en plus de $K$, $\omega_n$ et $\zeta$. Les signaux $x$ et $y$ sont en Volts.    
> Quelles expressions sont homogènes ?
>   - $(\frac{4}{T_e^2}+2\omega_n).z+\omega_n^2$
>   - $(\frac{4}{T_e^2}+2\omega_n^2).z^2+\omega_n^2$
>   - $(\frac{4}{T_e}+2\omega_n).z^2+\omega_n+ \frac{8}{T_e}$

On sait que le gain statique de ce second ordre vaut $G(p\equiv j\omega = 0)=K$ et  
qu'il est passe-bas. Donc pour les fréquences hautes $G(p\equiv j\omega = j\infty)=0$.
> - Donnez l'équivalent harmonique (en $\omega$) de $z$ : le fameux $z\equiv ?(\omega)$.  
>    Pour cela, rappelez-vous ce que fait l'opérateur avance $z$ sur une onde discrète pure $e^{i\omega\,k\,T_e}$
> - Donnez l'équivalent harmonique de $z$ en statique ($\omega=0$)
> - Donnez l'équivalent harmonique de $z$ aux hautes fréquences  
>   Attention au piège : quelle est la plus haute fréquence en discret ?
> - Vous pouvez donc calculer $G_d(z=1)$ et $G_d(z=-1)$ pour vérifier vos gains statiques et hautes fréquences.  


 Dans le tableau suivant, donnez les gains statiques et haute fréquence de ces F.T.,  
  lesquelles sont des candidats corrects qui peuvent correspondre au système continu du second ordre ?


| $G_d(z)$  | Gain statique | Gain HF | Homogène ? | Candidat correct ? |
|-------|---------------|---------|--------------------|----|
|$\frac{K}{z^2+ 2\zeta\omega_n\,z+{\omega_n}^2}$ |  |  |  |   |
|$K \frac{\omega_n^2.(z+1)^2}{(z-1)^2+ \frac{4\zeta\omega_n}{T_e}\,(z^2-1)+{\omega_n}^2.(z+1)^2}$ |  |  |  |   |
|$\frac{K (z+1)^2}{(z-1)+(z+1)^2}$ |  |  |  |    |




### Calcul de $G_d(z)$

> **Calculez $G_d(z)=G(G_t(z))$ sur votre copie papier AVEC VERIFICATIONS**  
> Vérifiez ainsi **sur votre copie papier** que :
>  - les termes sont homogènes entre eux 
>  - le gain statique vaut $K$ et le gain hautes fréquences vaut $0$
>
> Faites-le **A CHAQUE LIGNE DE CALCUL** pour profiter de la méthode au plus tôt...
>
> On préfèrera l'expression de $G_d(z)$ en $z$ plutôt que la formule en $z^{-1}$ 

Concluez en exprimant $G_d(z)$ sous la forme :

$$G_d(z) = K'. \frac{z^2 + b_1 z + b_2}{z^2 + a_1 z + a_2} = K'. \frac{1 + b_1 z^{-1} + b_2 z^{-2}}{1 + a_1 z^{-1} + a_2 z^{-2}}$$

En donnant les expressions analytiques des paramètres :  
$a_1 = f(K,T_e,\omega_n, \zeta)$, $a_2 = f(K,T_e,\omega_n, \zeta)$,  
$b_1 = f(K,T_e,\omega_n, \zeta)$, $b_2 = f(K,T_e,\omega_n, \zeta)$,
$K' = f(K,T_e,\omega_n, \zeta)$

Vérifiez leurs dimensions et une dernière fois que $G_d(1)=K$ et $G_d(-1)=0$ (éviter les erreurs de recopie !).

## 4 - Passage à la récurrence
---

Donnez **sur votre copie papier** la récurrence associée à la fonction de transfert 
$$G_d(z) = K'. \frac{1 + b_1 z^{-1} + b_2 z^{-2}}{1 + a_1 z^{-1} + a_2 z^{-2}}$$

En l'exprimant avec les paramètres $k'$, $a_n$, $b_m$ sous la forme : 

$$ y[k+1] = \pm a_1 \ldots \pm K' . (b_1 \ldots)$$


En régime permanent, on peut supposer que l'entrée $x[k]=x[k-1]=\ldots = x^\star$. 

Si le système est stable, alors la suite de valeur $(y[k])_{k\in\mathbb{Z}}$ possède une limite $y^\star$.

En utilisant le théorème des suites extraites : les suites $\left(y[k]\right)$, $\left(y[k-1]\right)$, $\ldots$ ont une même limite unique notée $y^\star$.

> On vérifie **sur la copie** cette récurrence :
>   - en vérifiant l'homogénéité des coefficients $a_i$ et $b_i$ sachant que $y$ et $x$ sont en Volts
>   - en vérifant qu'en régime statique $y^\star = K x^\star$ en introduisant la limite $y^\star$ et la constante $x^\star$ dans l'équation récurrente.   
>     Propriété des sommes de suites et multiplication de suistes par des constantes. 
     
Et oui ! Une erreur de signe est vite arrivée, mais vite corrigée.  
On rencontre dans la littérature (et les outils de calcul) les deux conventions de notation :
$$G_d(z) = K'. \frac{1 + b_1 z^{-1} + b_2 z^{-2}}{1 + a_1 z^{-1} + a_2 z^{-2}}$$
et  
$$G_d(z) = K'. \frac{1 + b_1 z^{-1} + b_2 z^{-2}}{1 - a_1 z^{-1} - a_2 z^{-2}}$$


## Vérifications en TP
---


Allez sur le [sujet de TP1](./tp1_continu_discret/tp1_bilineaire.ipynb) qui commence par vérifier numériquement pour des valeurs fixées de $\omega_n$ et $\zeta$ vos calculs.

